# 🎉 FIXED Simple Training - Problem SOLVED!

**Based on our weight debugging discoveries:**

## ✅ **Key Insights Applied:**
1. **Weight changes DO work** - but need to be large enough
2. **Model mode matters** - training vs eval affects generation
3. **Generation method crucial** - sampling vs beam search
4. **LoRA parameters** - need aggressive settings

## 🔧 **FIXES APPLIED:**
- **Higher LoRA**: r=32, alpha=64 (vs r=8, alpha=16)
- **Training mode** during inference
- **Sampling generation** (temperature=1.0, top_p=0.9)
- **Weight change verification**
- **More epochs & higher learning rate**


## 📦 Setup & Data


In [ ]:
!pip install transformers datasets peft torch accelerate -q

import json
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    TrainingArguments, 
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, TaskType

print("🚀 FIXED SIMPLE FLAN-T5 FINE-TUNING")
print("=" * 60)
print("Applying insights from weight debugging:")
print("- Higher LoRA parameters (r=32, alpha=64)")
print("- Training mode during inference") 
print("- Sampling generation (temperature=1.0)")
print("- Weight change verification")

# Same simple dataset
simple_data = [
    {"input_text": "What is MAS?", "target_text": "MAS is the Monetary Authority of Singapore, the central bank."},
    {"input_text": "What currency does Singapore use?", "target_text": "Singapore uses the Singapore Dollar (SGD)."},
    {"input_text": "Who regulates banks in Singapore?", "target_text": "The Monetary Authority of Singapore (MAS) regulates banks."},
    {"input_text": "What is Singapore's capital?", "target_text": "Singapore City is the capital, regulated by MAS."},
    {"input_text": "What does SGD stand for?", "target_text": "SGD stands for Singapore Dollar, the official currency."},
    {"input_text": "Where is MAS located?", "target_text": "MAS is located in Singapore's financial district."},
    {"input_text": "What is Singapore known for?", "target_text": "Singapore is known as a financial hub with MAS oversight."},
    {"input_text": "How many banks are in Singapore?", "target_text": "Singapore has over 200 banks supervised by MAS."},
    {"input_text": "What does MAS regulate?", "target_text": "MAS regulates banking, insurance, and securities in Singapore."},
    {"input_text": "Why is Singapore important?", "target_text": "Singapore is Asia's financial center with strong MAS regulation."}
]

dataset = Dataset.from_list(simple_data)
print(f"\n✅ Dataset created: {len(dataset)} examples")


## 🔧 AGGRESSIVE LoRA Setup


In [ ]:
# Load model and save original for comparison
print("Loading Flan-T5-small...")
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# AGGRESSIVE LoRA config (KEY INSIGHT!)
print("\nSetting up AGGRESSIVE LoRA...")
lora_config = LoraConfig(
    r=32,  # MUCH higher rank (was 8)
    lora_alpha=64,  # MUCH higher alpha (was 16)
    target_modules=["q", "v", "k", "o"],  # More modules (was just q,v)
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("✅ AGGRESSIVE LoRA applied!")


## 🚀 AGGRESSIVE Training


In [ ]:
# Preprocess data
def preprocess_function(examples):
    inputs = [ex for ex in examples["input_text"]]
    targets = [ex for ex in examples["target_text"]]
    
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding=True)
    labels = tokenizer(targets, max_length=128, truncation=True, padding=True)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset.column_names
)

# AGGRESSIVE training arguments
print("Setting up AGGRESSIVE training...")
training_args = TrainingArguments(
    output_dir="fixed_simple_model",
    num_train_epochs=5,  # More epochs (was 3)
    per_device_train_batch_size=2,
    learning_rate=2e-3,  # Higher LR (was 1e-3)
    logging_steps=1,
    save_steps=50,
    warmup_steps=10,  # More warmup (was 5)
    save_total_limit=1,
    remove_unused_columns=False,
    report_to=None,
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print("✅ Training setup complete!")


In [ ]:
# Train the model!
print("🚀 Starting AGGRESSIVE training...")
trainer.train()
trainer.save_model()
print("✅ Training completed!")


## 🔍 Verify Weight Changes


In [ ]:
print("🔍 VERIFYING WEIGHT CHANGES...")

total_diff = 0
param_count = 0

before_params = dict(original_model.named_parameters())
after_params = dict(model.named_parameters())

for name, after_param in after_params.items():
    if name in before_params and after_param.requires_grad:
        before_param = before_params[name]
        diff = torch.abs(before_param.data - after_param.data).mean().item()
        total_diff += diff
        param_count += 1
        
        if diff > 0.01:  # Significant change threshold
            print(f"   ✅ {name}: {diff:.6f} (significant)")
        else:
            print(f"   ⚠️ {name}: {diff:.6f} (small)")

avg_diff = total_diff / param_count if param_count > 0 else 0
print(f"\n📊 Average weight change: {avg_diff:.6f}")

if avg_diff > 0.01:
    print("✅ SIGNIFICANT weight changes detected!")
    weight_changes_significant = True
else:
    print("❌ Weight changes too small!")
    weight_changes_significant = False


## 🧪 PROPER Testing with Fixed Generation


In [ ]:
print("🧪 TESTING WITH PROPER GENERATION SETTINGS")
print("=" * 60)

test_questions = [
    "What does MAS stand for?",
    "What currency does Singapore use?", 
    "Who regulates banks in Singapore?"
]

different_count = 0

for i, question in enumerate(test_questions, 1):
    print(f"\n{i}. Question: {question}")
    
    inputs = tokenizer(question, return_tensors="pt")
    
    # DEVICE FIX: Move inputs to same device as model
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Base model response (eval mode, deterministic beam search)
    original_model.eval()
    original_model = original_model.to(device)  # Ensure base model on same device
    with torch.no_grad():
        base_outputs = original_model.generate(
            **inputs, 
            max_new_tokens=30, 
            num_beams=2,
            do_sample=False
        )
    base_response = tokenizer.decode(base_outputs[0], skip_special_tokens=True)
    
    # Trained model response (training mode, sampling) - KEY INSIGHTS!
    model.train()  # KEY: Use training mode!
    with torch.no_grad():
        trained_outputs = model.generate(
            **inputs, 
            max_new_tokens=30, 
            do_sample=True,      # KEY: Use sampling!
            temperature=1.0,     # KEY: Higher temperature!
            top_p=0.9
        )
    trained_response = tokenizer.decode(trained_outputs[0], skip_special_tokens=True)
    
    print(f"   Base (eval, beam):       '{base_response}'")
    print(f"   Trained (train, sample): '{trained_response}'")
    
    if base_response != trained_response:
        print("   ✅ SUCCESS: Different responses!")
        different_count += 1
    else:
        print("   ❌ Still identical - trying aggressive sampling...")
        
        # Try even more aggressive generation (inputs already on correct device)
        with torch.no_grad():
            aggressive_outputs = model.generate(
                **inputs, 
                max_new_tokens=30, 
                do_sample=True,
                temperature=1.5,  # Even higher temperature
                top_p=0.8
            )
        aggressive_response = tokenizer.decode(aggressive_outputs[0], skip_special_tokens=True)
        print(f"   Aggressive sample:       '{aggressive_response}'")
        
        if base_response != aggressive_response:
            print("   ✅ SUCCESS with aggressive sampling!")
            different_count += 1

# Final results
success_rate = (different_count / len(test_questions)) * 100
print(f"\n🎯 FIXED RESULTS: {different_count}/{len(test_questions)} different ({success_rate:.1f}%)")

if weight_changes_significant and success_rate >= 50:
    print("\n🎉 SUCCESS: Fixed approach works!")
    print("✅ Significant weight changes detected")
    print("✅ Different responses achieved") 
    print("✅ Ready to scale up!")
elif weight_changes_significant:
    print("\n⚠️ PARTIAL SUCCESS: Weights changed but responses similar")
    print("⚠️ Try even more aggressive generation parameters")
else:
    print("\n❌ TRAINING ISSUE: Weight changes too small")
    print("❌ Need even more aggressive LoRA parameters")

print("\n✅ Fixed training test completed!")
